In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
from Dataset.UEset import UEset
from models.vgg import vgg16_bn_tiny
from models.resnet import ResNet10

In [3]:
pre_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    ])
root = '/remote-home/share/MM_Ultrasound'

In [4]:
data = UEset(root, pre_transform = pre_transform, sub_list = [1,2,3,4,5],label=True)

In [5]:
loader = DataLoader(data,batch_size=256,shuffle=True,num_workers=4)

In [6]:
for imgs, label in loader:
    print(imgs.shape)

torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([256, 4, 3, 224, 224])
torch.Size([146, 4, 3, 224, 224])


In [15]:
from models.encoder import Res_Encoder
model = Res_Encoder()

In [31]:
class Res_Encoder(nn.Module):
    def __init__(self, input_dim=3, L=500, D=128, K=1):
        super(Res_Encoder, self).__init__()
        self.input_dim = input_dim
        self.L = L
        self.D = D
        self.K = K

        self.feature_extractor_part1 = ResNet10()

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(512 * 7 * 7, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

    def forward(self, x, debug=False):

        batch = x.shape[0]
        x = x.view(-1,3,224,224)
        print(x.shape)
        f = self.feature_extractor_part1(x)
        f = f.view(-1,512*7*7)
        print(f.shape)
        
        f = self.feature_extractor_part2(f)
        f = f.view(batch,4,-1)

    

        return f

In [32]:
input = torch.randn((128,4,3,224,224))

In [33]:
model = Res_Encoder()

In [34]:
output = model(input)

torch.Size([512, 3, 224, 224])
torch.Size([512, 25088])


In [35]:
output.shape

torch.Size([128, 4, 500])

In [36]:
f11 = output[:,0,:]

In [37]:
f11.shape

torch.Size([128, 500])

In [5]:
imgs, label = data[19]

In [9]:
img11, img21, img12, img22 = imgs

In [11]:
img11.shape

torch.Size([3, 224, 224])

In [13]:
model = vgg16_bn_tiny()

In [22]:
output = model(img11.unsqueeze(0))

In [25]:
len(output)

5

In [27]:
model2 = ResNet10()

In [29]:
model2(img11.unsqueeze(0)).shape

torch.Size([1, 512, 7, 7])

In [ ]:
class Res_Attention(nn.Module):
    def __init__(self, input_dim=3, L=500, D=128, K=1):
        super(Res_Attention, self).__init__()
        self.input_dim = input_dim
        self.L = L
        self.D = D
        self.K = K

        self.feature_extractor_part1 = ResNet10()

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(512 * 7 * 7, self.L),
            nn.ReLU(),
        )


## For InfoNCE

In [38]:
# 256 imgs with dim 512
f11 = torch.randn(256,512)
f21 = torch.randn(256,512)
f12 = torch.randn(256,512)
f22 = torch.randn(256,512)

In [39]:
criterion = nn.CrossEntropyLoss()

In [41]:
t = 0.01
# for data augmentation loss
l_pos_aug1 = torch.einsum('nc,nc->n',[f11,f12]).unsqueeze(-1)
l_pos_aug2 = torch.einsum('nc,nc->n',[f21,f22]).unsqueeze(-1)
l_neg_aug1 = torch.einsum('nc,ck->nk',[f11,f12.T])
l_neg_aug2 = torch.einsum('nc,ck->nk',[f21,f22.T])
logits_aug1 = torch.cat([l_pos_aug1,l_neg_aug1],dim=1)
logits_aug2 = torch.cat([l_pos_aug2,l_neg_aug2],dim=1)
labels = torch.zeros(logits_aug1.shape[0],dtype=torch.long)
aug_loss = criterion(logits_aug1,labels) + criterion(logits_aug2,labels)


In [42]:
labels.shape

torch.Size([256])

In [47]:
# for domain invariant loss
l_pos_d1 = torch.einsum('nc,nc->n',[f11,f21]).unsqueeze(-1)
l_pos_d2 = torch.einsum('nc,nc->n',[f12,f22]).unsqueeze(-1)
l_neg_d1 = torch.einsum('nc,ck->nk',[f11,f21.T])
l_neg_d2 = torch.einsum('nc,ck->nk',[f12,f22.T])
logits_d1 = torch.cat([l_pos_d1,l_neg_d1],dim=1)
logits_d2 = torch.cat([l_pos_d2,l_neg_d2],dim=1)
labels = torch.zeros(logits_d1.shape[0],dtype=torch.long)
d_loss = criterion(logits_d1,labels) + criterion(logits_d2,labels)

In [48]:
aug_loss

tensor(127.3748)

In [49]:
d_loss

tensor(130.3878)